In [16]:
# !pip install torch pandas scikit-learn transformers accelerate -U

In [1]:
# Réinstaller PyTorch et transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers tokenizers datasets accelerate

print("\n" + "="*60)
print("⚠️  ATTENTION : VOUS DEVEZ MAINTENANT REDÉMARRER LE RUNTIME !")
print("="*60)
print("\n👉 Menu : Runtime > Restart runtime")
print("👉 Ou utilisez le raccourci : Ctrl+M .")
print("\n🚫 N'exécutez AUCUNE autre cellule avant le redémarrage !")

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_cupti_cu11-11.8.87-py3-none-manylinux1_x86_64.whl (13.1 MB)
     ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/663.9 MB 214.6 MB/s eta 0:00:03
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.9 MB/s eta 0:00:00


KeyboardInterrupt: 

In [1]:
# This cell must be executed FIRST after restart
import torch
import transformers

print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda if torch.cuda.is_available() else "N/A")

# Simple test
try:
    x = torch.randn(2, 2)
    if torch.cuda.is_available():
        x = x.cuda()
    print(f"Test successful - Device: {x.device}")
    print("\nEverything works! You can now execute your code.")
except Exception as e:
    print(f"Error: {e}")

PyTorch version: 2.6.0+cu124
Transformers version: 4.55.0
CUDA available: True
CUDA version: 12.4
Test successful - Device: cuda:0

Everything works! You can now execute your code.


In [2]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm


In [3]:
# Version verification
import torch
import transformers
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Import test
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
print("Import successful!")

PyTorch version: 2.6.0+cu124
Transformers version: 4.55.0
CUDA available: True


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Import successful!


In [4]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# --- Constants ---
MODEL_NAME = "google/rembert"
LABEL_COLUMNS = ['political', 'racial/ethnic', 'religious', 'gender/sexual', 'other']
N_SPLITS = 5
MAX_LENGTH = 128

# --- Data Loading ---
# CORRECTION 1: 'path' was not defined
path = "/content/eng_train_2.csv"  # Replace with actual path
df = pd.read_csv(path)

# CORRECTION 2: Verify that columns exist
print(f"Available columns: {df.columns.tolist()}")

# CORRECTION 3: Handle missing values before conversion
df[LABEL_COLUMNS] = df[LABEL_COLUMNS].fillna(0).astype(float)
print(f"Dataset Shape: {df.shape}")
print(f"Label examples:\n{df[LABEL_COLUMNS].head()}")

# --- Additional Checks ---
# CORRECTION 4: Verify that text column exists
text_column = 'text'  # Adjust according to your dataset
if text_column not in df.columns:
    print(f"ERROR: Column '{text_column}' does not exist!")
    print(f"Available columns: {df.columns.tolist()}")
else:
    print(f"Text column found: {text_column}")
    print(f"Text example: {df[text_column].iloc[0][:100]}...")

# CORRECTION 5: Check label distribution
print("\nLabel distribution:")
print(df[LABEL_COLUMNS].sum())

Available columns: ['id', 'text', 'political', 'racial/ethnic', 'religious', 'gender/sexual', 'other']
Dataset Shape: (3222, 7)
Label examples:
   political  racial/ethnic  religious  gender/sexual  other
0        0.0            0.0        0.0            0.0    0.0
1        0.0            0.0        0.0            0.0    0.0
2        0.0            0.0        0.0            0.0    0.0
3        0.0            0.0        0.0            0.0    0.0
4        0.0            0.0        0.0            0.0    0.0
Text column found: text
Text example:  is defending imperialism in the dnd chat...

Label distribution:
political        1150.0
racial/ethnic     281.0
religious         112.0
gender/sexual      72.0
other             126.0
dtype: float64


In [5]:
# ---------------------------------------------
# 1. IMPORT LIBRARIES
# ---------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

# Make plots look nicer
sns.set(style="whitegrid")


# ---------------------------------------------
# 2. READ THE DATA
# ---------------------------------------------
path = "/content/eng_train_2.csv"
df = pd.read_csv(path)

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (3222, 7)


,id,text,political,racial/ethnic,religious,gender/sexual,other
0,eng_973938b90b0ff5d87d35a582f83f5c89,is defending imperialism in the dnd chat,0,0,0,0,0
1,eng_07dfd4600426caca6e2c5883fcbea9ea,Still playing with this. I am now following Ra...,0,0,0,0,0
2,eng_f14519ff2302b6cd47712073f13bc461,.senate.gov Theres 3 groups out there Republic...,0,0,0,0,0
3,eng_e48b7e7542faafa544ac57b64bc80daf,"""ABC MD, David Anderson, said the additional f...",0,0,0,0,0
4,eng_7c581fb77bce8033aeba3d6dbd6273eb,"""bad people"" I have some conservative values s...",0,0,0,0,0


Loading Data and Defining Constants

In [6]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# --- Constants ---
MODEL_NAME = "google/rembert"
LABEL_COLUMNS = ['political', 'racial/ethnic', 'religious', 'gender/sexual', 'other']
N_SPLITS = 5
MAX_LENGTH = 64  # Maximum sequence length (adjust if necessary)

# --- Data Loading ---
path = "/content/eng_train_2.csv"  # Define the path variable
df = pd.read_csv(path)  # Use the actual path to your file

# Convert labels to float for BCEWithLogitsLoss
df[LABEL_COLUMNS] = df[LABEL_COLUMNS].astype(float)
print(f"Dataset Shape: {df.shape}")

Dataset Shape: (3222, 7)


Definition of the PyTorch Dataset

In [7]:
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

# Initialisation du tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Evaluation Metric (Macro F1)

In [8]:
def compute_macro_f1(preds_logits, labels):
    # Model predictions are logits
    # For evaluation during training, use a default threshold (0.5)
    preds = (torch.sigmoid(torch.from_numpy(preds_logits)).numpy() > 0.5).astype(int)

    # Calculate Macro F1-score
    macro_f1 = f1_score(labels, preds, average='macro', zero_division=0)

    return {"macro_f1": macro_f1}

# Function for Trainer (requires a function that takes an EvalPrediction object)
def compute_metrics(p):
    return compute_macro_f1(p.predictions, p.label_ids)

5-Fold Cross-Validation and Logits Collection

In [9]:
os.environ["WANDB_DISABLED"] = "true"

In [10]:
# --- Clear GPU Memory ---
import torch
import gc

# Clear all cached memory
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# Reset peak memory stats
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.reset_accumulated_memory_stats()

print("GPU memory cleared!")
print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

GPU memory cleared!
Memory allocated: 0.00 GB
Memory cached: 0.00 GB


In [11]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# --- Constants ---
MODEL_NAME = "google/rembert"
LABEL_COLUMNS = ['political', 'racial/ethnic', 'religious', 'gender/sexual', 'other']
MAX_LENGTH = 64
TEST_SIZE = 0.2
RANDOM_STATE = 42

# --- Load Data ---
path = "/content/eng_train_2.csv"
df = pd.read_csv(path)
df[LABEL_COLUMNS] = df[LABEL_COLUMNS].astype(float)
print(f"Dataset Shape: {df.shape}")

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# --- Compute Metrics Function ---
def compute_macro_f1(preds_logits, labels):
    # Model predictions are logits
    # For evaluation during training, use a default threshold (0.5)
    preds = (torch.sigmoid(torch.from_numpy(preds_logits)).numpy() > 0.5).astype(int)

    # Calculate Binary F1-score
    f1 = f1_score(labels, preds, average='binary', zero_division=0)

    return {"f1": f1}

def compute_metrics(p):
    return compute_macro_f1(p.predictions, p.label_ids)

# --- Dictionary to store trained models and predictions ---
trained_models = {}
val_predictions_per_class = {}
val_labels_per_class = {}

# --- Train a separate model for each polarization class ---
for label_name in LABEL_COLUMNS:
    print(f"\n{'='*60}")
    print(f"TRAINING MODEL FOR: {label_name.upper()}")
    print(f"{'='*60}")

    # 1. Split data into train and validation
    train_df, val_df = train_test_split(
        df,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        stratify=df[label_name]
    )

    print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")
    print(f"Class distribution - Train: {train_df[label_name].value_counts().to_dict()}")
    print(f"Class distribution - Val: {val_df[label_name].value_counts().to_dict()}")

    # 2. Create datasets with single label
    train_dataset = PolarizationDataset(
        train_df['text'].tolist(),
        train_df[[label_name]].values,  # Single label column
        tokenizer,
        MAX_LENGTH
    )
    val_dataset = PolarizationDataset(
        val_df['text'].tolist(),
        val_df[[label_name]].values,  # Single label column
        tokenizer,
        MAX_LENGTH
    )

    # 3. Load Model (binary classification for this specific class)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=1,  # Binary classification: 1 output neuron
        problem_type="multi_label_classification"
    )

    # 4. Configure training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{label_name}',
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=4,
        learning_rate=3e-5,
        warmup_steps=500,
        weight_decay=0.1,
        logging_dir=f'./logs_{label_name}',
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=True
    )

    # 5. Create Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # 6. Training
    print(f"\nTraining {label_name} model...")
    trainer.train()

    # 7. Validation predictions
    val_predictions = trainer.predict(val_dataset)

    # Store predictions and labels
    val_predictions_per_class[label_name] = val_predictions.predictions
    val_labels_per_class[label_name] = val_predictions.label_ids

    # Calculate F1 score for this class
    preds = (torch.sigmoid(torch.from_numpy(val_predictions.predictions)).numpy() > 0.5).astype(int)
    class_f1 = f1_score(val_predictions.label_ids, preds, average='binary', zero_division=0)
    print(f"\n{label_name} - Validation F1-Score: {class_f1:.4f}")

    # 8. Save the trained model
    trained_models[label_name] = trainer
    model.save_pretrained(f'./model_{label_name}')

    # 9. Free GPU memory
    del model
    torch.cuda.empty_cache()

    print(f"\n--- Completed training for {label_name} ---")

# --- Aggregate results across all classes ---
print("\n" + "="*60)
print("FINAL VALIDATION RESULTS")
print("="*60)

# Combine all predictions
all_val_logits = np.concatenate([val_predictions_per_class[label] for label in LABEL_COLUMNS], axis=1)
all_val_labels = np.concatenate([val_labels_per_class[label] for label in LABEL_COLUMNS], axis=1)

# Calculate overall macro F1
final_preds = (torch.sigmoid(torch.from_numpy(all_val_logits)).numpy() > 0.5).astype(int)
macro_f1_overall = f1_score(all_val_labels, final_preds, average='macro', zero_division=0)

print("\nIndividual Class F1-Scores:")
for label_name in LABEL_COLUMNS:
    preds = (torch.sigmoid(torch.from_numpy(val_predictions_per_class[label_name])).numpy() > 0.5).astype(int)
    class_f1 = f1_score(val_labels_per_class[label_name], preds, average='binary', zero_division=0)
    print(f"  {label_name}: {class_f1:.4f}")

print(f"\nOverall Macro F1-Score: {macro_f1_overall:.4f}")
print("\nAll models saved successfully!")

Dataset Shape: (3222, 7)

TRAINING MODEL FOR: POLITICAL
Train size: 2577, Validation size: 645
Class distribution - Train: {0.0: 1657, 1.0: 920}
Class distribution - Val: {0.0: 415, 1.0: 230}


Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3863991236.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Training political model...


Epoch,Training Loss,Validation Loss,F1
1,0.589100,0.447204,0.724638
2,0.442100,0.514095,0.515337
3,0.401200,0.655388,0.647887



political - Validation F1-Score: 0.7246

--- Completed training for political ---

TRAINING MODEL FOR: RACIAL/ETHNIC
Train size: 2577, Validation size: 645
Class distribution - Train: {0.0: 2352, 1.0: 225}
Class distribution - Val: {0.0: 589, 1.0: 56}


Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3863991236.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Training racial/ethnic model...


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 14.56 GiB of which 17.75 MiB is free. Process 29378 has 14.54 GiB memory in use. Of the allocated memory 12.73 GiB is allocated by PyTorch, and 1.68 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import torch

# --- Configuration ---
N_SPLITS = 5
LABEL_COLUMNS = ['political', 'racial/ethnic', 'religious', 'gender/sexual', 'other']

# Dictionary to store OOF predictions for each class
all_class_oof_logits = {label: [] for label in LABEL_COLUMNS}
all_class_oof_labels = {label: [] for label in LABEL_COLUMNS}

# --- Train a separate model for each polarization class ---
for label_name in LABEL_COLUMNS:
    print(f"\n{'='*60}")
    print(f"TRAINING MODEL FOR: {label_name.upper()}")
    print(f"{'='*60}")

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

    for fold, (train_index, val_index) in enumerate(kf.split(df)):
        print(f"\n--- Starting Fold {fold+1}/{N_SPLITS} for {label_name} ---")

        # 1. Prepare fold data with single label
        train_df = df.iloc[train_index]
        val_df = df.iloc[val_index]

        train_dataset = PolarizationDataset(
            train_df['text'].tolist(),
            train_df[[label_name]].values,  # Single label column
            tokenizer,
            MAX_LENGTH
        )
        val_dataset = PolarizationDataset(
            val_df['text'].tolist(),
            val_df[[label_name]].values,  # Single label column
            tokenizer,
            MAX_LENGTH
        )

        # 2. Load Model (binary classification for this specific class)
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=1,  # Binary classification: 1 output neuron
            problem_type="multi_label_classification"
        )

        # 3. Configure training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_{label_name}_fold_{fold}',
            num_train_epochs=5,
            per_device_train_batch_size=2,
            gradient_accumulation_steps=8,
            per_device_eval_batch_size=4,
            learning_rate=3e-5,
            warmup_steps=500,
            weight_decay=0.1,
            logging_dir=f'./logs_{label_name}',
            logging_steps=100,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="macro_f1",
            greater_is_better=True,
            fp16=True
        )

        # 4. Create Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # 5. Training
        print(f"Training {label_name} model...")
        trainer.train()

        # 6. OOF Predictions on validation set
        oof_predictions = trainer.predict(val_dataset)

        # Store logits for this class
        all_class_oof_logits[label_name].append(oof_predictions.predictions)
        all_class_oof_labels[label_name].append(oof_predictions.label_ids)

        # 7. Free GPU memory
        del model
        torch.cuda.empty_cache()

    print(f"\n--- Completed all folds for {label_name} ---")

# --- Aggregate OOF results for each class ---
print("\n" + "="*60)
print("AGGREGATING OUT-OF-FOLD PREDICTIONS")
print("="*60)

oof_logits_per_class = {}
oof_labels_per_class = {}

for label_name in LABEL_COLUMNS:
    oof_logits_per_class[label_name] = np.concatenate(all_class_oof_logits[label_name], axis=0)
    oof_labels_per_class[label_name] = np.concatenate(all_class_oof_labels[label_name], axis=0)

    # Calculate F1 score for this class
    preds = (torch.sigmoid(torch.from_numpy(oof_logits_per_class[label_name])).numpy() > 0.5).astype(int)
    class_f1 = f1_score(oof_labels_per_class[label_name], preds, average='binary', zero_division=0)
    print(f"{label_name}: F1-Score = {class_f1:.4f}")

# Combine all predictions for final macro F1 calculation
all_oof_logits = np.concatenate([oof_logits_per_class[label] for label in LABEL_COLUMNS], axis=1)
all_oof_labels = np.concatenate([oof_labels_per_class[label] for label in LABEL_COLUMNS], axis=1)

# Calculate overall macro F1
final_preds = (torch.sigmoid(torch.from_numpy(all_oof_logits)).numpy() > 0.5).astype(int)
macro_f1_overall = f1_score(all_oof_labels, final_preds, average='macro', zero_division=0)
print(f"\nOverall Macro F1-Score: {macro_f1_overall:.4f}")

Optimisation du Seuil par Classe (Le point clé)

In [ ]:
# ---------------------------------------------
# 3. LABEL DISTRIBUTION
# ---------------------------------------------
label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

label_counts = df[label_cols].sum().sort_values(ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title("Label Distribution")
plt.ylabel("Number of Samples")
plt.xticks(rotation=45)
plt.show()


In [ ]:
label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

for col in label_cols:
    count = df[col].value_counts()
    print(f"{col}: {count}")

In [ ]:
label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

for col in label_cols:
    count = df[col].sum()
    print(f"{col}: {count} rows")


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Define colors for each label type and slight variations for classes 0 and 1
color_scheme = {
    "political":       ("#9ecae1", "#08519c"),  # light blue, dark blue
    "racial/ethnic":   ("#a1d99b", "#238b45"),  # light green, dark green
    "religious":       ("#cbc9e2", "#6a51a3"),  # light purple, dark purple
    "gender/sexual":   ("#fcbba1", "#cb181d"),  # light red, dark red
    "other":           ("#fdd0a2", "#e6550d")   # light orange, dark orange
}

label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

for col in label_cols:
    print(f"Generating colored wordclouds for: {col}")

    # Fetch custom colors
    color_0, color_1 = color_scheme[col]

    # Create separate word lists
    text_0 = " ".join(df[df[col] == 0]["text"].astype(str))
    text_1 = " ".join(df[df[col] == 1]["text"].astype(str))

    # Custom color functions
    def color_func_0(*args, **kwargs):
        return color_0

    def color_func_1(*args, **kwargs):
        return color_1

    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(18, 10), dpi=300)
    fig.suptitle(f"WordClouds for '{col}'", fontsize=22)

    # WordCloud for class 0
    wc0 = WordCloud(width=1000, height=800, background_color='white',
                    color_func=color_func_0).generate(text_0)
    axes[0].imshow(wc0, interpolation='bilinear')
    axes[0].set_title(f"{col} = 0 (Non-class)", fontsize=18)
    axes[0].axis('off')

    # WordCloud for class 1
    wc1 = WordCloud(width=1000, height=800, background_color='white',
                    color_func=color_func_1).generate(text_1)
    axes[1].imshow(wc1, interpolation='bilinear')
    axes[1].set_title(f"{col} = 1 (Class)", fontsize=18)
    axes[1].axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from tqdm import tqdm

# ============================================================
# 1. Dataset
# ============================================================

class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

        self.texts = df["text"].tolist()
        self.labels = df[["political", "racial/ethnic", "religious", "gender/sexual", "other"]].values.astype(float)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.float)
        }


# ============================================================
# 2. Focal Loss with Class Weights
# ============================================================

class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha  # tensor of size [num_labels]
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        bce = F.binary_cross_entropy_with_logits(
            logits, targets, reduction="none"
        )

        probs = torch.sigmoid(logits)
        pt = torch.where(targets == 1, probs, 1 - probs)

        focal = (self.alpha * (1 - pt) ** self.gamma * bce)

        if self.reduction == "mean":
            return focal.mean()
        else:
            return focal.sum()


# ============================================================
# 3. Load Data
# ============================================================

df = pd.read_csv("/content/gdrive/MyDrive/subtask2/eng-train-task2.csv")
df = df.rename(columns={"label":"other"})  # only if needed

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print("Sizes -> train/val/test:", len(train_df), len(val_df), len(test_df))

# ============================================================
# 4. Compute Class Weights
# ============================================================

label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

pos_counts = train_df[label_cols].sum(axis=0).values
neg_counts = len(train_df) - pos_counts
weights = torch.tensor(neg_counts / pos_counts, dtype=torch.float)

print("Class weights:", weights)

# ============================================================
# 5. Load Model + Tokenizer
# ============================================================

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=5,
    problem_type="multi_label_classification"
).to(device)

# ============================================================
# 6. Dataloaders
# ============================================================

train_ds = TextDataset(train_df, tokenizer)
val_ds = TextDataset(val_df, tokenizer)
test_ds = TextDataset(test_df, tokenizer)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

# ============================================================
# 7. Loss + Optimizer
# ============================================================

criterion = FocalLoss(alpha=weights.to(device), gamma=2.0)
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 4

# ============================================================
# 8. Train / Eval Functions
# ============================================================

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0

    for batch in tqdm(loader, desc="Train"):
        input_ids = batch["input_ids"].to(device)
        attn = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attn)
        logits = outputs.logits

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


def eval_one_epoch(model, loader):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Eval"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask=attn).logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(probs)
            all_labels.append(labels.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    macro_f1 = f1_score(all_labels, (all_preds > 0.5), average="macro")

    return total_loss / len(loader), macro_f1, all_preds, all_labels


# ============================================================
# 9. Training Loop
# ============================================================

for epoch in range(epochs):
    print(f"\n=== Epoch {epoch+1}/{epochs} ===")
    train_loss = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_f1, _, _ = eval_one_epoch(model, val_loader)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Macro F1: {val_f1:.4f}")

# ============================================================
# 10. Threshold Search on Val Set
# ============================================================

def find_best_thresholds(preds, labels):
    thresholds = {}
    for i, col in enumerate(label_cols):
        best_f1 = 0
        best_t = 0.5
        for t in np.linspace(0.05, 0.95, 20):
            f1 = f1_score(labels[:, i], (preds[:, i] > t))
            if f1 > best_f1:
                best_f1 = f1
                best_t = t
        thresholds[col] = np.float64(best_t)
    return thresholds

_, _, val_preds, val_labels = eval_one_epoch(model, val_loader)
thresholds = find_best_thresholds(val_preds, val_labels)
print("Best thresholds:", thresholds)

# ============================================================
# 11. FINAL TEST EVALUATION
# ============================================================

def apply_thresholds(preds, thresholds):
    binarized = np.zeros_like(preds)
    for i, col in enumerate(label_cols):
        binarized[:, i] = preds[:, i] > thresholds[col]
    return binarized

_, _, test_preds, test_labels = eval_one_epoch(model, test_loader)
bin_preds = apply_thresholds(test_preds, thresholds)

macro_f1 = f1_score(test_labels, bin_preds, average="macro")
micro_f1 = f1_score(test_labels, bin_preds, average="micro")

print("\n=== FINAL TEST SCORES ===")
print("Macro F1:", macro_f1)
print("Micro F1:", micro_f1)


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

### 1. LOAD THE DEV SET (THE ONE YOU MUST SUBMIT)
df_dev = pd.read_csv("/content/gdrive/MyDrive/subtask2/eng-dev-task2.csv")

# If your dataset column is not exactly "text", modify here:
texts = df_dev["text"].tolist()

### 2. DATASET FOR PREDICTION
class PredDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=256):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
        }

pred_dataset = PredDataset(texts, tokenizer)
pred_loader = DataLoader(pred_dataset, batch_size=16, shuffle=False)

### 3. GET RAW PREDICTIONS
all_logits = []

with torch.no_grad():
    for batch in tqdm(pred_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)

        all_logits.extend(probs.cpu().numpy())

all_logits = np.array(all_logits)

### 4. APPLY BEST THRESHOLDS YOU FOUND
thresholds = {
    "political": 0.381578947368421,
    "racial/ethnic": 0.33421052631578946,
    "religious": 0.381578947368421,
    "gender/sexual": 0.381578947368421,
    "other": 0.14473684210526316
}

label_order = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

preds_binary = np.zeros_like(all_logits)

for i, label in enumerate(label_order):
    preds_binary[:, i] = (all_logits[:, i] >= thresholds[label]).astype(int)

### 5. CREATE SUBMISSION FILE
submission = pd.DataFrame({
    "id": df_dev["id"],
    "political": preds_binary[:, 0],
    "racial/ethnic": preds_binary[:, 1],
    "religious": preds_binary[:, 2],
    "gender/sexual": preds_binary[:, 3],
    "other": preds_binary[:, 4],
})

submission.to_csv("submission.csv", index=False)

print("Saved submission.csv!")


In [ ]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

########################################
# 1. LOAD TRAIN + DEV DATASETS
########################################

train_df = pd.read_csv("/content/gdrive/MyDrive/subtask2/eng-train-task2.csv")
dev_df   = pd.read_csv("/content/gdrive/MyDrive/subtask2/eng-dev-task2.csv")

TEXT_COL = "text"
LABEL_COLS = ["political","racial/ethnic","religious","gender/sexual","other"]
NUM_LABELS = len(LABEL_COLS)

########################################
# 2. STRATIFY TARGET FOR MULTILABEL
########################################

# Convert multilabel → string for stratification (best practice)
train_df["stratify_label"] = train_df[LABEL_COLS].astype(str).agg("".join, axis=1)

########################################
# 3. Dataset Class
########################################

class HateDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df[TEXT_COL].tolist()
        self.labels = df[LABEL_COLS].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.float)
        }

########################################
# 4. Model + Loss Function
########################################

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

def create_model():
    model = AutoModelForSequenceClassification.from_pretrained(
        "cardiffnlp/twitter-roberta-base-hate-latest",
        num_labels=NUM_LABELS,
        problem_type="multi_label_classification",
        ignore_mismatched_sizes=True # Added this line to fix the size mismatch error
    )
    return model.to(DEVICE)

criterion = torch.nn.BCEWithLogitsLoss()

########################################
# 5. Train One Fold
########################################

def train_fold(model, train_loader, val_loader, epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5)

    for epoch in range(epochs):
        model.train()
        losses = []
        for batch in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
            optimizer.zero_grad()
            batch = {k: v.to(DEVICE) for k,v in batch.items()}
            outputs = model(**batch)
            loss = criterion(outputs.logits, batch["labels"])
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        print(f"Train Loss: {np.mean(losses):.4f}")

    # Validation predictions
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in val_loader:
            labels = batch["labels"].numpy()
            batch = {k: v.to(DEVICE) for k,v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits.sigmoid().cpu().numpy()

            preds.append(logits)
            trues.append(labels)

    preds = np.vstack(preds)
    trues = np.vstack(trues)

    # threshold = 0.3 (best for Cardiff hate model)
    bin_preds = (preds > 0.3).astype(int)

    f1 = f1_score(trues, bin_preds, average="micro")
    print("Validation Micro F1:", f1)

    return model, preds, f1

########################################
# 6. K-Fold Cross Validation
########################################

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_preds = []
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df["stratify_label"])):
    print(f"\n===== FOLD {fold+1} =====")
    model = create_model()

    train_data = train_df.iloc[train_idx]
    val_data   = train_df.iloc[val_idx]

    train_ds = HateDataset(train_data, tokenizer)
    val_ds   = HateDataset(val_data, tokenizer)

    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)

    model, preds, f1 = train_fold(model, train_loader, val_loader)
    fold_preds.append(preds)
    fold_scores.append(f1)

print("\n=== MEAN CV SCORE:", np.mean(fold_scores), "===")

########################################
# 7. Train FINAL MODEL on ALL TRAIN DATA
########################################

final_model = create_model()
full_ds = HateDataset(train_df, tokenizer)
full_loader = DataLoader(full_ds, batch_size=16, shuffle=True)

optimizer = AdamW(final_model.parameters(), lr=2e-5)

for epoch in range(3):
    final_model.train()
    losses = []
    for batch in tqdm(full_loader, desc=f"Final Train Epoch {epoch+1}"):
        optimizer.zero_grad()
        batch = {k:v.to(DEVICE) for k,v in batch.items()}
        outputs = final_model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print("Final Train Loss:", np.mean(losses))

########################################
# 8. INFERENCE ON DEV
########################################

dev_ds = HateDataset(dev_df, tokenizer)
dev_loader = DataLoader(dev_ds, batch_size=16, shuffle=False)

final_model.eval()
all_preds = []

with torch.no_grad():
    for batch in tqdm(dev_loader):
        batch = {k: v.to(DEVICE) for k,v in batch.items()}
        logits = final_model(**batch).logits.sigmoid().cpu().numpy()
        all_preds.append(logits)

all_preds = np.vstack(all_preds)
binary_preds = (all_preds > 0.3).astype(int)

########################################
# 9. SAVE SUBMISSION
########################################

out = dev_df.copy()
for i, col in enumerate(LABEL_COLS):
    out[col] = binary_preds[:, i]

out.to_csv("polar_submission.csv", index=False)
print("Saved → polar_submission.csv")

In [ ]:
# ---------- PREDICT ON DEV + SAVE SUBMISSION ----------
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm

# 1) device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2) label order (must match model output order)
label_cols = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]

# 3) load dev file (path where you uploaded it)
dev_path = "/content/gdrive/MyDrive/subtask2/eng-dev-task2.csv"   # adjust if different
dev_df = pd.read_csv(dev_path)
print("Dev loaded:", dev_df.shape)

# ensure id and text exist; try common alternatives
if "id" not in dev_df.columns:
    # try first column as id
    dev_df = dev_df.rename(columns={dev_df.columns[0]: "id"})
if "text" not in dev_df.columns:
    # try common alternative names
    for candidate in ["tweet", "content", "message"]:
        if candidate in dev_df.columns:
            dev_df = dev_df.rename(columns={candidate: "text"})
            break
# final fallback: convert second column to text if no 'text'
if "text" not in dev_df.columns:
    dev_df = dev_df.rename(columns={dev_df.columns[1]: "text"})

# ensure text is string
dev_df["text"] = dev_df["text"].fillna("").astype(str)

# 4) load tokenizer + model (use in-memory if available, otherwise from disk)
# Try to use existing variables if they exist in the notebook:
try:
    tokenizer  # noqa: F821
    print("Using tokenizer from environment")
except Exception:
    # fallback: try common saved tokenizer names
    for path in ["./final_roberta_multilabel", "./final_model", "./cardiff_final", "./final"]:
        if os.path.isdir(path):
            try:
                tokenizer = AutoTokenizer.from_pretrained(path)
                print("Loaded tokenizer from", path)
                break
            except Exception:
                tokenizer = None
    if "tokenizer" not in globals() or tokenizer is None:
        # last resort: load cardiff tokenizer (fast)
        tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
        print("Loaded tokenizer from HuggingFace cardiffnlp/twitter-roberta-base-hate-latest")

try:
    final_model  # noqa: F821
    model = final_model
    print("Using final_model from environment")
except Exception:
    # try loading from common save locations
    model = None
    for path in ["./final_roberta_multilabel", "./final_model", "./cardiff_final", "./final"]:
        if os.path.isdir(path):
            try:
                model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=len(label_cols), problem_type="multi_label_classification")
                print("Loaded model from", path)
                break
            except Exception as e:
                print("Could not load model from", path, ":", e)
    if model is None:
        # fallback to the Cardiff checkpoint and re-init classification head
        model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest",
                                                                   num_labels=len(label_cols),
                                                                   problem_type="multi_label_classification")
        print("Loaded cardiffnlp model and reinitialized head")

model = model.to(device)
model.eval()

# 5) Dataset + DataLoader for dev
class DevDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        txt = self.texts[idx]
        enc = self.tokenizer(txt,
                             padding="max_length",
                             truncation=True,
                             max_length=self.max_len,
                             return_tensors="pt")
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0)
        }

dev_dataset = DevDataset(dev_df["text"].tolist(), tokenizer, max_len=128)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

# 6) thresholds: use existing 'thresholds' if present, else fallback to reasonable defaults
try:
    thresholds  # noqa: F821
    print("Using thresholds from environment")
except Exception:
    # fallback thresholds you reported earlier (adjust if you prefer)
    thresholds = {
        "political": 0.381578947368421,
        "racial/ethnic": 0.33421052631578946,
        "religious": 0.381578947368421,
        "gender/sexual": 0.381578947368421,
        "other": 0.14473684210526316
    }
    print("Using fallback thresholds:", thresholds)

# align thresholds into array in label_cols order
th_arr = np.array([thresholds[c] for c in label_cols], dtype=float)

# 7) predict probabilities
all_probs = []
with torch.no_grad():
    for batch in tqdm(dev_loader, desc="Predict"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        out = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = out.logits
        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)
all_probs = np.vstack(all_probs)  # shape (N_dev, 5)
print("Preds shape:", all_probs.shape)

# 8) binarize according to thresholds
bin_preds = (all_probs > th_arr).astype(int)

# 9) create submission dataframe with exact required columns
sub_df = pd.DataFrame()
sub_df["id"] = dev_df["id"].astype(str).values  # ensure id preserved as string
for i, col in enumerate(label_cols):
    sub_df[col] = bin_preds[:, i].astype(int)

# 10) save CSV (no extra index or weird nested columns)
out_path = "./polar_submissiion.csv"
sub_df.to_csv(out_path, index=False)
print("Saved submission to", out_path)
print(sub_df.head(5))


Subtask 3: Manifestation Identification